In [2]:
pip install -r requirements.txt


  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2

In [1]:
import random
import numpy as np

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


NameError: name 'torch' is not defined

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional
import os

class HiRALayer(nn.Module):
    def __init__(
        self,
        in_features: int,
        out_features: int,
        r: int = 32,
        lora_alpha: int = 32,
        lora_dropout: float = 0.1,
    ):
        super().__init__()
        self.r = r
        self.lora_alpha = lora_alpha
        self.scaling = lora_alpha / r
        
        self.lora_A = nn.Parameter(torch.zeros(in_features, r))
        self.lora_B = nn.Parameter(torch.randn(r, out_features))
        
        nn.init.zeros_(self.lora_A)
        nn.init.kaiming_uniform_(self.lora_B, a=0)
        
        self.lora_dropout = nn.Dropout(p=lora_dropout) if lora_dropout > 0 else nn.Identity()
        
    def forward(self, x: torch.Tensor, W0: torch.Tensor) -> torch.Tensor:
        result = F.linear(x, W0)
        
        lora_update = self.lora_A @ self.lora_B  # [in_features, out_features]
        
        hadamard_update = W0.T * lora_update  # [in_features, out_features]
        
        result += self.lora_dropout(x @ hadamard_update) * self.scaling
        
        return result


class HiRALinear(nn.Module):
    def __init__(self, linear_layer: nn.Linear, r: int = 32, lora_alpha: int = 32):
        super().__init__()
        self.linear = linear_layer
        self.hira = HiRALayer(
            in_features=linear_layer.in_features,
            out_features=linear_layer.out_features,
            r=r,
            lora_alpha=lora_alpha
        )
        self.linear.weight.requires_grad = False
        if self.linear.bias is not None:
            self.linear.bias.requires_grad = False
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.hira(x, self.linear.weight)




In [3]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch.nn as nn

def apply_hira_to_model(model, r=32, lora_alpha=32, target_modules=['q_lin', 'k_lin', 'v_lin', 'out_lin', 'ffn.lin1', 'ffn.lin2']):

    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            if any(target in name for target in target_modules):
                parent_name = '.'.join(name.split('.')[:-1])
                attr_name = name.split('.')[-1]
                
                if parent_name:
                    parent_module = model.get_submodule(parent_name)
                else:
                    parent_module = model
                
                hira_layer = HiRALinear(module, r=r, lora_alpha=lora_alpha)
                setattr(parent_module, attr_name, hira_layer)
                
                print(f"Applied HiRA to: {name}")
    
    return model

def get_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    return trainable_params, all_params, 100 * trainable_params / all_params


/Users/gaozhiyuan/ECE685_Final_Project/.conda/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

def load_sst2_data(tokenizer, max_length=128):
    """8:1:1 """
    raw = load_dataset("glue", "sst2")  # 有 train / validation / test[web:110]

    train_valid = raw["train"]
    train_valid = train_valid.shuffle(seed=42)
    n = len(train_valid)
    n_train = int(0.8 * n)
    n_val = int(0.1 * n)
    n_test = n - n_train - n_val

    train_dataset = train_valid.select(range(n_train))
    val_dataset   = train_valid.select(range(n_train, n_train + n_val))
    test_dataset  = train_valid.select(range(n_train + n_val, n))

    def preprocess_function(examples):
        enc = tokenizer(
            examples["sentence"],
            truncation=True,
            max_length=max_length
        )
        enc["labels"] = examples["label"]
        return enc

    train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
    val_dataset   = val_dataset.map(preprocess_function,   batched=True, remove_columns=val_dataset.column_names)
    test_dataset  = test_dataset.map(preprocess_function,  batched=True, remove_columns=test_dataset.column_names)

    return {
        "train": train_dataset,
        "validation": val_dataset,
        "test": test_dataset,
    }



def load_imdb_data(tokenizer, max_length=256):

    raw = load_dataset("imdb")  

    train_full = raw["train"]               
    train_full = train_full.shuffle(seed=42)
    n = len(train_full)
    n_train = int(0.8 * n)
    n_val = int(0.1 * n)
    n_test = n - n_train - n_val

    train_dataset = train_full.select(range(n_train))
    val_dataset   = train_full.select(range(n_train, n_train + n_val))
    test_dataset  = train_full.select(range(n_train + n_val, n))

    def preprocess_function(examples):
        enc = tokenizer(
            examples["text"],
            truncation=True,
            max_length=max_length,
        )
        enc["labels"] = examples["label"]
        return enc

    train_dataset = train_dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=train_dataset.column_names,
    )
    val_dataset = val_dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=val_dataset.column_names,
    )
    test_dataset = test_dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=test_dataset.column_names,
    )

    return {
        "train": train_dataset,
        "validation": val_dataset,
        "test": test_dataset,
    }



# After changing

In [5]:
import os
import time
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import (
    DistilBertForSequenceClassification,
    DistilBertTokenizer,
    DataCollatorWithPadding,
    get_linear_schedule_with_warmup,
)
from tqdm import tqdm
import evaluate
from datasets import load_dataset




def get_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for p in model.parameters():
        num = p.numel()
        all_params += num
        if p.requires_grad:
            trainable_params += num
    percentage = 100.0 * trainable_params / all_params if all_params > 0 else 0.0
    return trainable_params, all_params, percentage


def get_model_sparsity(model, threshold: float = 1e-3) -> float:
    total_elems = 0
    small_elems = 0
    for p in model.parameters():
        if p is None:
            continue
        data = p.detach()
        total_elems += data.numel()
        small_elems += (data.abs() < threshold).sum().item()
    if total_elems == 0:
        return 0.0
    return small_elems / total_elems

def train_hira_model(
    dataset_name: str = "sst2",
    model_name: str = "distilbert-base-uncased",
    r: int = 32,
    lora_alpha: int = 32,
    num_epochs: int = 3,
    batch_size: int = 16,
    learning_rate: float = 2e-5,
    warmup_steps: int = 100,
    max_length: int = 128,
    output_dir: str = "./results_hira",
):

    device = torch.device(
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
    print(f"[{dataset_name}][r={r}] Using device: {device}")

    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        return_tensors="pt",
    )

    num_labels = 2
    model = DistilBertForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
    )
    model = apply_hira_to_model(model, r=r, lora_alpha=lora_alpha)
    model = model.to(device)

    trainable_params, all_params, percentage = get_trainable_parameters(model)
    print(
        f"[{dataset_name}][r={r}] Trainable params: {trainable_params:,} || "
        f"All params: {all_params:,} || Trainable%: {percentage:.4f}%"
    )

    if dataset_name == "sst2":
        dataset = load_sst2_data(tokenizer, max_length)
    elif dataset_name == "imdb":
        dataset = load_imdb_data(tokenizer, max_length)
    else:
        raise ValueError(f"Unknown dataset_name: {dataset_name}")

    train_dataloader = DataLoader(
        dataset["train"],
        batch_size=batch_size,
        shuffle=True,
        collate_fn=data_collator,
    )
    val_split_name = "validation" if "validation" in dataset else "test"
    val_dataloader = DataLoader(
        dataset[val_split_name],
        batch_size=batch_size,
        collate_fn=data_collator,
    )
    test_dataloader = DataLoader(
        dataset["test"],
        batch_size=batch_size,
        collate_fn=data_collator,
    )

    optimizer = AdamW(
        [p for p in model.parameters() if p.requires_grad],
        lr=learning_rate,
    )
    total_steps = len(train_dataloader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps,
    )

    os.makedirs(output_dir, exist_ok=True)

    epoch_times = []
    total_train_time = 0.0
    best_val_accuracy = 0.0
    best_val_f1 = 0.0
    best_epoch = -1

    for epoch in range(num_epochs):
        start_t = time.perf_counter()

        model.train()
        total_loss = 0.0
        progress_bar = tqdm(
            train_dataloader,
            desc=f"[{dataset_name}][r={r}] Epoch {epoch + 1}/{num_epochs}",
        )

        for batch in progress_bar:
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            total_loss += loss.item()
            progress_bar.set_postfix({"loss": loss.item()})

        avg_train_loss = total_loss / len(train_dataloader)

        acc_metric = evaluate.load("accuracy")
        f1_metric = evaluate.load("f1")
        model.eval()
        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)
            preds = torch.argmax(outputs.logits, dim=-1)
            acc_metric.add_batch(predictions=preds, references=batch["labels"])
            f1_metric.add_batch(predictions=preds, references=batch["labels"])

        val_acc = acc_metric.compute()["accuracy"]
        val_f1 = f1_metric.compute()["f1"]

        end_t = time.perf_counter()
        epoch_time = end_t - start_t
        epoch_times.append(epoch_time)
        total_train_time += epoch_time

        print(
            f"[{dataset_name}][r={r}] Epoch {epoch + 1} | "
            f"train_loss={avg_train_loss:.4f} | "
            f"val_acc={val_acc:.4f} | "
            f"val_f1={val_f1:.4f} | "
            f"time/epoch={epoch_time:.2f}s"
        )

        if val_f1 > best_val_f1:
            best_val_accuracy = val_acc
            best_val_f1 = val_f1
            if best_epoch == -1:
                best_epoch = epoch + 1

            save_path = os.path.join(output_dir, f"best_model_r{r}.pt")
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "accuracy": best_val_accuracy,
                },
                save_path,
            )
            print(
                f"[{dataset_name}][r={r}] Saved best model with "
                f"val_acc={best_val_accuracy:.4f} to {save_path}"
            )

    avg_time_per_epoch = sum(epoch_times) / len(epoch_times)
    print(
        f"[{dataset_name}][r={r}] Training completed! "
        f"best_val_acc={best_val_accuracy:.4f}, "
        f"avg_time/epoch={avg_time_per_epoch:.2f}s, "
        f"converge_epoch={best_epoch}, "
        f"trainable_params={trainable_params}, "
        f"trainable_ratio={percentage:.4f}%"
    )

    acc_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")
    model.eval()
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=-1)
        acc_metric.add_batch(predictions=preds, references=batch["labels"])
        f1_metric.add_batch(predictions=preds, references=batch["labels"])

    test_acc = acc_metric.compute()["accuracy"]
    test_f1 = f1_metric.compute()["f1"]
    print(f"[{dataset_name}][r={r}] Final test accuracy: {test_acc:.4f}, test_f1={test_f1:.4f}")

    sparsity = get_model_sparsity(model, threshold=1e-3)

    return {
        "dataset": dataset_name,
        "r": r,
        "best_val_acc": best_val_accuracy,
        "best_val_f1": best_val_f1,
        "test_acc": test_acc,
        "test_f1": test_f1,
        "avg_time_per_epoch": avg_time_per_epoch,
        "total_train_time": total_train_time,
        "converge_epoch": best_epoch,
        "trainable_params": trainable_params,
        "total_params": all_params,
        "trainable_ratio": percentage,   
        "sparsity": sparsity,            
    }


def format_int_with_commas(x: int) -> str:
    return f"{x:,}"


def generate_summary_table(results):
    results_sorted = sorted(
        results,
        key=lambda d: d["test_acc"],
        reverse=True,
    )

    header = (
        "| Rank | Trainable Params / Total | Ratio | Val F1 | Val Acc | "
        "Test F1 | Test Acc | Sparsity (<1e−3) | Train Time (s) |\n"
    )
    sep = (
        "| ---- | ------------------------ | ----- | ------ | ------- | "
        "------- | -------- | ----------------- | -------------- |\n"
    )

    lines = [header, sep]
    for rank, res in enumerate(results_sorted, start=1):
        trainable = format_int_with_commas(res["trainable_params"])
        total = f"{res['total_params']/1e6:.1f}M"
        ratio = f"{res['trainable_ratio']:.2f}%"
        val_f1 = f"{res['best_val_f1']:.4f}"
        val_acc = f"{res['best_val_acc']:.4f}"
        test_f1 = f"{res['test_f1']:.4f}"
        test_acc = f"{res['test_acc']:.4f}"
        sparsity = f"{res['sparsity']*100:.2f}%"
        train_time = f"{res['total_train_time']:.2f}"

        line = (
            f"| {rank} | {trainable} / {total} | {ratio} | "
            f"{val_f1} | {val_acc} | {test_f1} | {test_acc} | "
            f"{sparsity} | {train_time} |\n"
        )
        lines.append(line)

    table_md = "".join(lines)
    print("\nResult Summary Table:\n")
    print(table_md)


In [6]:

print("=" * 50)
print("Training HiRA on SST-2 with different ranks")
print("=" * 50)

results = []
for r in [2, 4, 8, 16]:
    print("\n" + "-" * 30)
    print(f"HiRA with rank r={r}")
    print("-" * 30)
    res = train_hira_model(
        dataset_name="sst2",
        model_name="distilbert-base-uncased",
        r=r,
        lora_alpha=32,
        num_epochs=10,        
        batch_size=16,
        learning_rate=2e-5,  
        warmup_steps=100,
        max_length=128,
        output_dir="./results_hira",
    )
    results.append(res)

generate_summary_table(results)

print("\nSummary over ranks:")
for res in results:
    print(
        f"r={res['r']}: "
        f"val_acc={res['best_val_acc']:.4f}, "
        f"test_acc={res['test_acc']:.4f}, "
        f"avg_time/epoch={res['avg_time_per_epoch']:.2f}s, "
        f"converge_epoch={res['converge_epoch']}, "
        f"trainable={res['trainable_params']} ({res['trainable_ratio']:.4f}%)"
    )



Training HiRA on SST-2 with different ranks

------------------------------
HiRA with rank r=2
------------------------------
[sst2][r=2] Using device: mps


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Applied HiRA to: distilbert.transformer.layer.0.attention.q_lin
Applied HiRA to: distilbert.transformer.layer.0.attention.k_lin
Applied HiRA to: distilbert.transformer.layer.0.attention.v_lin
Applied HiRA to: distilbert.transformer.layer.0.attention.out_lin
Applied HiRA to: distilbert.transformer.layer.0.ffn.lin1
Applied HiRA to: distilbert.transformer.layer.0.ffn.lin2
Applied HiRA to: distilbert.transformer.layer.1.attention.q_lin
Applied HiRA to: distilbert.transformer.layer.1.attention.k_lin
Applied HiRA to: distilbert.transformer.layer.1.attention.v_lin
Applied HiRA to: distilbert.transformer.layer.1.attention.out_lin
Applied HiRA to: distilbert.transformer.layer.1.ffn.lin1
Applied HiRA to: distilbert.transformer.layer.1.ffn.lin2
Applied HiRA to: distilbert.transformer.layer.2.attention.q_lin
Applied HiRA to: distilbert.transformer.layer.2.attention.k_lin
Applied HiRA to: distilbert.transformer.layer.2.attention.v_lin
Applied HiRA to: distilbert.transformer.layer.2.attention.out_li

[sst2][r=2] Epoch 1/10: 100%|██████████| 3368/3368 [04:55<00:00, 11.40it/s, loss=0.373] 


[sst2][r=2] Epoch 1 | train_loss=0.3480 | val_acc=0.9024 | val_f1=0.9131 | time/epoch=319.12s
[sst2][r=2] Saved best model with val_acc=0.9024 to ./results_hira/best_model_r2.pt


...

[sst2][r=16] Epoch 8 | train_loss=0.0911 | val_acc=0.9361 | val_f1=0.9429 | time/epoch=344.36s
[sst2][r=16] Saved best model with val_acc=0.9361 to ./results_hira/best_model_r16.pt


[sst2][r=16] Epoch 9/10: 100%|██████████| 3368/3368 [05:25<00:00, 10.33it/s, loss=0.0782] 


[sst2][r=16] Epoch 9 | train_loss=0.0848 | val_acc=0.9360 | val_f1=0.9429 | time/epoch=342.16s


[sst2][r=16] Epoch 10/10: 100%|██████████| 3368/3368 [05:32<00:00, 10.14it/s, loss=0.173]  


[sst2][r=16] Epoch 10 | train_loss=0.0830 | val_acc=0.9367 | val_f1=0.9436 | time/epoch=348.33s
[sst2][r=16] Saved best model with val_acc=0.9367 to ./results_hira/best_model_r16.pt
[sst2][r=16] Training completed! best_val_acc=0.9367, avg_time/epoch=340.71s, converge_epoch=1, trainable_params=25773314, trainable_ratio=37.7453%
[sst2][r=16] Final test accuracy: 0.9381, test_f1=0.9458

Result Summary Table:

| Rank | Trainable Params / Total | Ratio | Val F1 | Val Acc | Test F1 | Test Acc | Sparsity (<1e−3) | Train Time (s) |
| ---- | ------------------------ | ----- | ------ | ------- | ------- | -------- | ----------------- | -------------- |
| 1 | 25,109,762 / 67.6M | 37.13% | 0.9432 | 0.9363 | 0.9462 | 0.9387 | 1.79% | 3325.13 |
| 2 | 24,612,098 / 67.1M | 36.67% | 0.9441 | 0.9373 | 0.9458 | 0.9382 | 1.74% | 3341.53 |
| 3 | 24,777,986 / 67.3M | 36.82% | 0.9442 | 0.9375 | 0.9459 | 0.9382 | 1.76% | 3384.06 |
| 4 | 25,773,314 / 68.3M | 37.75% | 0.9436 | 0.9367 | 0.9458 | 0.9381 | 1.84% 

In [7]:
generate_summary_table(results)



Result Summary Table:

| Rank | Trainable Params / Total | Ratio | Val F1 | Val Acc | Test F1 | Test Acc | Sparsity (<1e−3) | Train Time (s) |
| ---- | ------------------------ | ----- | ------ | ------- | ------- | -------- | ----------------- | -------------- |
| 1 | 25,109,762 / 67.6M | 37.13% | 0.9432 | 0.9363 | 0.9462 | 0.9387 | 1.79% | 3325.13 |
| 2 | 24,612,098 / 67.1M | 36.67% | 0.9441 | 0.9373 | 0.9458 | 0.9382 | 1.74% | 3341.53 |
| 3 | 24,777,986 / 67.3M | 36.82% | 0.9442 | 0.9375 | 0.9459 | 0.9382 | 1.76% | 3384.06 |
| 4 | 25,773,314 / 68.3M | 37.75% | 0.9436 | 0.9367 | 0.9458 | 0.9381 | 1.84% | 3407.09 |



In [8]:

print("=" * 50)
print("Training HiRA on SST-2 with different ranks")
print("=" * 50)

imdb_results = []
for r in [2, 4, 8, 16]:
    print("\n" + "-" * 30)
    print(f"HiRA with rank r={r}")
    print("-" * 30)
    res = train_hira_model(
        dataset_name="imdb",
        model_name="distilbert-base-uncased",
        r=r,
        lora_alpha=32,
        num_epochs=10,        
        batch_size=16,
        learning_rate=2e-5,  
        warmup_steps=100,
        max_length=128,
        output_dir="./results_hira",
    )
    imdb_results.append(res)

generate_summary_table(imdb_results)

print("\nSummary over ranks:")
for res in imdb_results:
    print(
        f"r={res['r']}: "
        f"val_acc={res['best_val_acc']:.4f}, "
        f"test_acc={res['test_acc']:.4f}, "
        f"avg_time/epoch={res['avg_time_per_epoch']:.2f}s, "
        f"converge_epoch={res['converge_epoch']}, "
        f"trainable={res['trainable_params']} ({res['trainable_ratio']:.4f}%)"
    )





Training HiRA on SST-2 with different ranks

------------------------------
HiRA with rank r=2
------------------------------
[imdb][r=2] Using device: mps


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Applied HiRA to: distilbert.transformer.layer.0.attention.q_lin
Applied HiRA to: distilbert.transformer.layer.0.attention.k_lin
Applied HiRA to: distilbert.transformer.layer.0.attention.v_lin
Applied HiRA to: distilbert.transformer.layer.0.attention.out_lin
Applied HiRA to: distilbert.transformer.layer.0.ffn.lin1
Applied HiRA to: distilbert.transformer.layer.0.ffn.lin2
Applied HiRA to: distilbert.transformer.layer.1.attention.q_lin
Applied HiRA to: distilbert.transformer.layer.1.attention.k_lin
Applied HiRA to: distilbert.transformer.layer.1.attention.v_lin
Applied HiRA to: distilbert.transformer.layer.1.attention.out_lin
Applied HiRA to: distilbert.transformer.layer.1.ffn.lin1
Applied HiRA to: distilbert.transformer.layer.1.ffn.lin2
Applied HiRA to: distilbert.transformer.layer.2.attention.q_lin
Applied HiRA to: distilbert.transformer.layer.2.attention.k_lin
Applied HiRA to: distilbert.transformer.layer.2.attention.v_lin
Applied HiRA to: distilbert.transformer.layer.2.attention.out_li

[imdb][r=2] Epoch 1/10: 100%|██████████| 1250/1250 [04:33<00:00,  4.58it/s, loss=0.399]


[imdb][r=2] Epoch 1 | train_loss=0.4822 | val_acc=0.8212 | val_f1=0.8158 | time/epoch=299.35s
[imdb][r=2] Saved best model with val_acc=0.8212 to ./results_hira/best_model_r2.pt


...

[imdb][r=16] Epoch 8 | train_loss=0.1549 | val_acc=0.8432 | val_f1=0.8447 | time/epoch=253.54s


[imdb][r=16] Epoch 9/10: 100%|██████████| 1250/1250 [1:00:30<00:00,  2.90s/it, loss=0.151]    


[imdb][r=16] Epoch 9 | train_loss=0.1439 | val_acc=0.8444 | val_f1=0.8457 | time/epoch=248.47s


[imdb][r=16] Epoch 10/10: 100%|██████████| 1250/1250 [52:38<00:00,  2.53s/it, loss=0.289]     


[imdb][r=16] Epoch 10 | train_loss=0.1355 | val_acc=0.8448 | val_f1=0.8453 | time/epoch=248.98s
[imdb][r=16] Training completed! best_val_acc=0.8448, avg_time/epoch=251.93s, converge_epoch=1, trainable_params=25773314, trainable_ratio=37.7453%
[imdb][r=16] Final test accuracy: 0.8424, test_f1=0.8483

Result Summary Table:

| Rank | Trainable Params / Total | Ratio | Val F1 | Val Acc | Test F1 | Test Acc | Sparsity (<1e−3) | Train Time (s) |
| ---- | ------------------------ | ----- | ------ | ------- | ------- | -------- | ----------------- | -------------- |
| 1 | 24,777,986 / 67.3M | 36.82% | 0.8475 | 0.8460 | 0.8526 | 0.8472 | 1.77% | 3565.19 |
| 2 | 24,612,098 / 67.1M | 36.67% | 0.8513 | 0.8492 | 0.8523 | 0.8460 | 1.75% | 3328.82 |
| 3 | 25,109,762 / 67.6M | 37.13% | 0.8434 | 0.8412 | 0.8507 | 0.8448 | 1.80% | 3285.02 |
| 4 | 25,773,314 / 68.3M | 37.75% | 0.8469 | 0.8448 | 0.8483 | 0.8424 | 1.88% | 2519.27 |


Summary over ranks:
r=2: val_acc=0.8492, test_acc=0.8460, avg_time/epoch

# Before changing

In [ ]:
import os
import time
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import (
    DistilBertForSequenceClassification,
    DistilBertTokenizer,
    DataCollatorWithPadding,
    get_linear_schedule_with_warmup,
)
from tqdm import tqdm
import evaluate


def train_hira_model(
    dataset_name: str = "sst2",           
    model_name: str = "distilbert-base-uncased",
    r: int = 32,                        
    lora_alpha: int = 32,
    num_epochs: int = 10,              
    batch_size: int = 16,
    learning_rate: float = 1e-3,
    warmup_steps: int = 100,
    max_length: int = 128,
    output_dir: str = "./results_hira",
):

    device = torch.device(
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
    print(f"[{dataset_name}][r={r}] Using device: {device}")

    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        return_tensors="pt",
    )

    num_labels = 2
    model = DistilBertForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
    )
    model = apply_hira_to_model(model, r=r, lora_alpha=lora_alpha)
    model = model.to(device)

    trainable_params, all_params, percentage = get_trainable_parameters(model)
    print(
        f"[{dataset_name}][r={r}] Trainable params: {trainable_params:,} || "
        f"All params: {all_params:,} || Trainable%: {percentage:.4f}%"
    )

    if dataset_name == "sst2":
        dataset = load_sst2_data(tokenizer, max_length)
    elif dataset_name == "imdb":
        dataset = load_imdb_data(tokenizer, max_length)
    else:
        raise ValueError(f"Unknown dataset_name: {dataset_name}")

    train_dataloader = DataLoader(
        dataset["train"],
        batch_size=batch_size,
        shuffle=True,
        collate_fn=data_collator,
    )
    val_dataloader = DataLoader(
        dataset["validation"],
        batch_size=batch_size,
        collate_fn=data_collator,
    )
    test_dataloader = DataLoader(
        dataset["test"],
        batch_size=batch_size,
        collate_fn=data_collator,
    )

    optimizer = AdamW(
        [p for p in model.parameters() if p.requires_grad],
        lr=learning_rate,
    )
    total_steps = len(train_dataloader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps,
    )

    os.makedirs(output_dir, exist_ok=True)

    epoch_times = []
    best_val_accuracy = 0.0
    best_epoch = -1

    for epoch in range(num_epochs):
        start_t = time.perf_counter()

        model.train()
        total_loss = 0.0
        progress_bar = tqdm(
            train_dataloader,
            desc=f"[{dataset_name}][r={r}] Epoch {epoch + 1}/{num_epochs}",
        )

        for batch in progress_bar:
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            total_loss += loss.item()
            progress_bar.set_postfix({"loss": loss.item()})

        avg_train_loss = total_loss / len(train_dataloader)

        model.eval()
        metric = evaluate.load("accuracy")
        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)
            preds = torch.argmax(outputs.logits, dim=-1)
            metric.add_batch(predictions=preds, references=batch["labels"])
        val_acc = metric.compute()["accuracy"]

        end_t = time.perf_counter()
        epoch_time = end_t - start_t
        epoch_times.append(epoch_time)

        print(
            f"[{dataset_name}][r={r}] Epoch {epoch + 1} | "
            f"train_loss={avg_train_loss:.4f} | "
            f"val_acc={val_acc:.4f} | "
            f"time/epoch={epoch_time:.2f}s"
        )

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            if best_epoch == -1:
                best_epoch = epoch + 1 

            save_path = os.path.join(output_dir, f"best_model_r{r}.pt")
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "accuracy": best_val_accuracy,
                },
                save_path,
            )
            print(
                f"[{dataset_name}][r={r}] Saved best model with "
                f"val_acc={best_val_accuracy:.4f} to {save_path}"
            )

    avg_time_per_epoch = sum(epoch_times) / len(epoch_times)
    print(
        f"[{dataset_name}][r={r}] Training completed! "
        f"best_val_acc={best_val_accuracy:.4f}, "
        f"avg_time/epoch={avg_time_per_epoch:.2f}s, "
        f"converge_epoch={best_epoch}, "
        f"trainable_params={trainable_params}, "
        f"trainable_ratio={percentage:.4f}%"
    )

    model.eval()
    metric = evaluate.load("accuracy")
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=-1)
        metric.add_batch(predictions=preds, references=batch["labels"])
    test_acc = metric.compute()["accuracy"]
    print(f"[{dataset_name}][r={r}] Final test accuracy: {test_acc:.4f}")

    return {
        "dataset": dataset_name,
        "r": r,
        "best_val_acc": best_val_accuracy,
        "test_acc": test_acc,
        "avg_time_per_epoch": avg_time_per_epoch,
        "converge_epoch": best_epoch,
        "trainable_params": trainable_params,
        "trainable_ratio": percentage,
    }


In [30]:
print("=" * 50)
print("Training HiRA on SST-2 with different ranks")
print("=" * 50)

results = []
for r in [2, 4, 8, 16]:
    print("\n" + "-" * 30)
    print(f"HiRA with rank r={r}")
    print("-" * 30)
    res = train_hira_model(
        dataset_name='sst2',
        r=r,
        num_epochs=10,
        batch_size=16,
        learning_rate=1e-3,
        max_length=128,
        output_dir=f'./results_hira_sst2_r{r}',
    )
    results.append(res)

print("\nSummary over ranks:")
for res in results:
    print(
        f"r={res['r']}: "
        f"val_acc={res['best_val_acc']:.4f}, "
        f"test_acc={res['test_acc']:.4f}, "
        f"avg_time/epoch={res['avg_time_per_epoch']:.2f}s, "
        f"converge_epoch={res['converge_epoch']}, "
        f"trainable={res['trainable_params']} ({res['trainable_ratio']:.4f}%)"
    )


Training HiRA on SST-2 with different ranks

------------------------------
HiRA with rank r=2
------------------------------
[sst2][r=2] Using device: mps


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Applied HiRA to: distilbert.transformer.layer.0.attention.q_lin
Applied HiRA to: distilbert.transformer.layer.0.attention.k_lin
Applied HiRA to: distilbert.transformer.layer.0.attention.v_lin
Applied HiRA to: distilbert.transformer.layer.0.attention.out_lin
Applied HiRA to: distilbert.transformer.layer.0.ffn.lin1
Applied HiRA to: distilbert.transformer.layer.0.ffn.lin2
Applied HiRA to: distilbert.transformer.layer.1.attention.q_lin
Applied HiRA to: distilbert.transformer.layer.1.attention.k_lin
Applied HiRA to: distilbert.transformer.layer.1.attention.v_lin
Applied HiRA to: distilbert.transformer.layer.1.attention.out_lin
Applied HiRA to: distilbert.transformer.layer.1.ffn.lin1
Applied HiRA to: distilbert.transformer.layer.1.ffn.lin2
Applied HiRA to: distilbert.transformer.layer.2.attention.q_lin
Applied HiRA to: distilbert.transformer.layer.2.attention.k_lin
Applied HiRA to: distilbert.transformer.layer.2.attention.v_lin
Applied HiRA to: distilbert.transformer.layer.2.attention.out_li

[sst2][r=2] Epoch 1/10: 100%|██████████| 3368/3368 [05:29<00:00, 10.23it/s, loss=0.106] 


[sst2][r=2] Epoch 1 | train_loss=0.2740 | val_acc=0.9180 | time/epoch=353.79s
[sst2][r=2] Saved best model with val_acc=0.9180 to ./results_hira_sst2_r2/best_model_r2.pt


...

[sst2][r=16] Epoch 8 | train_loss=0.0063 | val_acc=0.9315 | time/epoch=398.23s
[sst2][r=16] Saved best model with val_acc=0.9315 to ./results_hira_sst2_r16/best_model_r16.pt


[sst2][r=16] Epoch 9/10: 100%|██████████| 3368/3368 [06:18<00:00,  8.89it/s, loss=0.00283] 


[sst2][r=16] Epoch 9 | train_loss=0.0047 | val_acc=0.9307 | time/epoch=398.27s


[sst2][r=16] Epoch 10/10: 100%|██████████| 3368/3368 [06:17<00:00,  8.93it/s, loss=0]       


[sst2][r=16] Epoch 10 | train_loss=0.0028 | val_acc=0.9311 | time/epoch=396.70s
[sst2][r=16] Training completed! best_val_acc=0.9315, avg_time/epoch=383.89s, converge_epoch=1, trainable_params=25773314, trainable_ratio=37.7453%
[sst2][r=16] Final test accuracy: 0.9313

Summary over ranks:
r=2: val_acc=0.9317, test_acc=0.9314, avg_time/epoch=354.09s, converge_epoch=1, trainable=24612098 (36.6683%)
r=4: val_acc=0.9326, test_acc=0.9317, avg_time/epoch=369.75s, converge_epoch=1, trainable=24777986 (36.8244%)
r=8: val_acc=0.9311, test_acc=0.9348, avg_time/epoch=370.70s, converge_epoch=1, trainable=25109762 (37.1344%)
r=16: val_acc=0.9315, test_acc=0.9313, avg_time/epoch=383.89s, converge_epoch=1, trainable=25773314 (37.7453%)


In [33]:
for r in [2, 4, 8, 16]:
    res = train_hira_model(
        dataset_name="imdb",
        r=r,
        num_epochs=10,
        batch_size=8,
        learning_rate=1e-3,
        max_length=256,
        output_dir=f"./results_hira_imdb_r{r}",
    )


[imdb][r=2] Using device: mps


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Applied HiRA to: distilbert.transformer.layer.0.attention.q_lin
Applied HiRA to: distilbert.transformer.layer.0.attention.k_lin
Applied HiRA to: distilbert.transformer.layer.0.attention.v_lin
Applied HiRA to: distilbert.transformer.layer.0.attention.out_lin
Applied HiRA to: distilbert.transformer.layer.0.ffn.lin1
Applied HiRA to: distilbert.transformer.layer.0.ffn.lin2
Applied HiRA to: distilbert.transformer.layer.1.attention.q_lin
Applied HiRA to: distilbert.transformer.layer.1.attention.k_lin
Applied HiRA to: distilbert.transformer.layer.1.attention.v_lin
Applied HiRA to: distilbert.transformer.layer.1.attention.out_lin
Applied HiRA to: distilbert.transformer.layer.1.ffn.lin1
Applied HiRA to: distilbert.transformer.layer.1.ffn.lin2
Applied HiRA to: distilbert.transformer.layer.2.attention.q_lin
Applied HiRA to: distilbert.transformer.layer.2.attention.k_lin
Applied HiRA to: distilbert.transformer.layer.2.attention.v_lin
Applied HiRA to: distilbert.transformer.layer.2.attention.out_li

[imdb][r=2] Epoch 1/10: 100%|██████████| 2500/2500 [11:02<00:00,  3.77it/s, loss=0.441]  


[imdb][r=2] Epoch 1 | train_loss=0.3759 | val_acc=0.8572 | time/epoch=690.73s
[imdb][r=2] Saved best model with val_acc=0.8572 to ./results_hira_imdb_r2/best_model_r2.pt


...

[imdb][r=16] Epoch 8 | train_loss=0.0012 | val_acc=0.8464 | time/epoch=745.77s


[imdb][r=16] Epoch 9/10: 100%|██████████| 2500/2500 [12:25<00:00,  3.35it/s, loss=0]       


[imdb][r=16] Epoch 9 | train_loss=0.0000 | val_acc=0.8436 | time/epoch=778.97s


[imdb][r=16] Epoch 10/10: 100%|██████████| 2500/2500 [11:52<00:00,  3.51it/s, loss=0]       


[imdb][r=16] Epoch 10 | train_loss=0.0000 | val_acc=0.8436 | time/epoch=744.97s
[imdb][r=16] Training completed! best_val_acc=0.8572, avg_time/epoch=737.97s, converge_epoch=1, trainable_params=25773314, trainable_ratio=37.7453%
[imdb][r=16] Final test accuracy: 0.8504
